In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import DBSCAN, KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
origin_df = pd.read_csv("../../data/customer_segments.csv")
df = origin_df.copy()
origin_df.head()

In [ ]:
df.info()

In [ ]:
# 1. 히스토그램 (분포 확인)
df.hist(bins=30, figsize=(15, 10), color="skyblue")
plt.suptitle("Feature Distributions")
plt.show()

# 2. 박스플롯 (이상치 확인 - 스케일링 전 필수)
plt.figure(figsize=(12, 6))
sns.boxplot(data=df.drop(columns=["Region"]))  # 범주형 제외
plt.xticks(rotation=45)
plt.title("Outlier Check")
plt.show()

# 3. 상관관계 히트맵
encoder = LabelEncoder()
df["Region"] = encoder.fit_transform(df["Region"])
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap="RdBu", fmt=".2f")
plt.title("Feature Correlation")
plt.show()

# 4. DBSCAN을 위한 K-distance Graph
# (반드시 스케일링된 데이터를 사용해야 합니다)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop(columns=["Region"]))

neigh = NearestNeighbors(n_neighbors=5)  # min_samples와 유사하게 설정
nbrs = neigh.fit(df_scaled)
distances, indices = nbrs.kneighbors(df_scaled)
distances = np.sort(distances[:, 4], axis=0)  # 5번째 이웃과의 거리

plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.title("K-distance Graph for DBSCAN Epsilon")
plt.ylabel("Epsilon (Distance)")
plt.xlabel("Data Points (sorted)")
plt.grid(True)
plt.show()

In [ ]:
df_features = df.drop(["Customer_ID", "Region"], axis=1)

# 아이디, 범주형 제외하고 스케일링
scaled_data = scaler.fit_transform(df_features)
df_scaled = pd.DataFrame(scaled_data, columns=df_features.columns)

df_scaled.hist(bins=30, figsize=(15, 10), color="skyblue")
plt.suptitle("Feature Distributions")
plt.show()

# 2. 박스플롯 (이상치 확인 - 스케일링 전 필수)
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_features)  # 범주형 제외
plt.xticks(rotation=45)
plt.title("Outlier Check")
plt.show()

# 3. 상관관계 히트맵
plt.figure(figsize=(10, 8))
sns.heatmap(df_features.corr(), annot=True, cmap="RdBu", fmt=".2f")
plt.title("Feature Correlation")
plt.show()

# 4. DBSCAN을 위한 K-distance Graph
# (반드시 스케일링된 데이터를 사용해야 합니다)
neigh = NearestNeighbors(n_neighbors=5)  # min_samples와 유사하게 설정
nbrs = neigh.fit(df_scaled)
distances, indices = nbrs.kneighbors(df_scaled)
distances = np.sort(distances[:, 4], axis=0)  # 5번째 이웃과의 거리

plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.title("K-distance Graph for DBSCAN Epsilon")
plt.ylabel("Epsilon (Distance)")
plt.xlabel("Data Points (sorted)")
plt.grid(True)
plt.show()


In [ ]:
# 1. 모델 선언 (K-distance graph 근거)
# eps는 1.5로 시작해서 결과 보고 조절하세요.
dbscan = DBSCAN(eps=2.2, min_samples=5)

# 2. 학습 및 라벨 추출
# 데이터프레임이든 넘파이 배열이든 상관없이 들어갑니다.
clusters = dbscan.fit_predict(df_scaled)

print("이상치 데이터 갯수 : ", (clusters == -1).sum())

set(clusters)

df_scaled["clusters"] = clusters
df["clusters"] = clusters

pair_col = [("Frequency_90D", "Monetary_90D"), ("Avg_Basket", "Monetary_90D")]

for x, y in pair_col:
    plt.scatter(
        df_scaled[x], df_scaled[y], alpha=0.5, c=df_scaled["clusters"], cmap="tab10"
    )

    plt.xlabel(x.split("_")[0])
    plt.ylabel(y.split("_")[0])
    plt.title("DBSCAN Clustering: Frequency vs Monetary")
    plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
# df_scaled가 아닌 원본 데이터(df)를 사용하는 것이 해석에 유리합니다.
sns.boxplot(x="clusters", y="Monetary_90D", data=df)
plt.title("Monetary Distribution by Cluster")
plt.show()

In [ ]:
# 1. 로그 변환이 필요한 컬럼들 (쏠림이 심한 것들)
log_features = [
    "Monetary_90D",
    "Frequency_90D",
    "Recency_Days",
    "Web_Visits_30D",
    "App_Sessions_30D",
]

# 2. 로그 변환이 필요 없는 컬럼들 (이미 분포가 골고루 퍼진 것들)
normal_features = ["Avg_Basket", "Discount_Ratio", "Return_Ratio", "Review_Score"]

# 3. 새로운 데이터프레임 구성
X_final = pd.DataFrame()

for col in log_features:
    X_final[col + "_log"] = np.log1p(df[col])  # 로그 변환 적용

for col in normal_features:
    X_final[col] = df[col]  # 그대로 가져옴

print(X_final)

# 4. 전체 스케일링 (중요!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(
    X_final
)  # 이제 이 데이터를 kmeans.fit(X_scaled)에 넣습니다.

kmeans = KMeans(n_clusters=4, random_state=3333)
kmeans.fit(X_scaled)

In [ ]:
print(f"Expected features: {kmeans.n_features_in_}")

In [ ]:
df_scaled["clusters"] = kmeans.labels_
df_scaled.head()

In [ ]:
kmeans.predict(df_scaled)

In [ ]:
df_scaled["clusters"].mean()

In [ ]:
plt.figure(figsize=(12, 10))

# x에 Frequency, y에 Monetary를 넣습니다.
sns.scatterplot(
    x="Frequency_90D",
    y="Monetary_90D",
    hue="clusters",
    data=df_scaled,
    palette="viridis",
    s=50,
    alpha=0.6,
    legend="full",
)

# 중심점(Centroids) 표시
# 주의: kmeans.cluster_centers_의 인덱스는 features 리스트의 순서와 같습니다.
# 만약 Frequency가 1번째, Monetary가 2번째 컬럼이었다면 아래와 같이 지정합니다.
plt.scatter(
    kmeans.cluster_centers_[:, 1],
    kmeans.cluster_centers_[:, 2],
    s=300,
    c="red",
    marker="*",
    label="Centroids",
)

plt.title("K-Means Clustering: Frequency vs Monetary")
plt.show()

In [ ]:
features = [
    "Recency_Days",
    "Frequency_90D",
    "Monetary_90D",
    "Avg_Basket",
    "Discount_Ratio",
    "Return_Ratio",
    "Web_Visits_30D",
    "App_Sessions_30D",
    "Review_Score",
]

df_melted = df_scaled.melt(
    id_vars=["clusters"], value_vars=features, var_name="Feature", value_name="Value"
)

plt.figure(figsize=(15, 7))
sns.lineplot(
    x="Feature", y="Value", hue="clusters", data=df_melted, palette="tab10", marker="o"
)

plt.title("Snake Plot: Customer Segmentation Profile")
plt.xticks(rotation=45)
plt.axhline(0, color="red", linestyle="--", alpha=0.3)  # 평균선(0) 표시
plt.grid(axis="x", alpha=0.3)
plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()